In [15]:
import numpy as np


#The main over-arching strucutre of the system, where the forest contains a number of trees set as n_estimator
#Also contains the max depth and the minimum samples per split
class RandomForest:
    #Initialize the constants of the random forest
    def __init__(self, n_estimator=100, max_depth=None, min_samples_split=2):
        self.n_estimator = n_estimator
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.trees = []
    #Iterates through each tree number, creates a decision tree and populates with randomly sampled points - with replacement
    #Then train/fit the decision tree on that sampled data, and append to the forest
    def fit(self, X, y):
        for _ in range(self.n_estimator):
            tree = DecisionTree(max_depth=self.max_depth, min_samples_split=self.min_samples_split)
            indices = np.random.choice(len(X), len(X), replace=True)
            tree.fit(X[indices], y[indices])
            self.trees.append(tree)

    #Creates a np-array that matches dim of the trees, iterates through the trees and predicts based on trees average
    def predict(self, X):
        predictions = np.zeros((len(X), len(self.trees)))
        for i, tree in enumerate(self.trees):
            predictions[:, i] = tree.predict(X)
        return np.mean(predictions, axis=1)


#Building block of each decision tree
class Node:
    #Initialize all constants of the Node
    def __init__ (self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature #which feature to split on
        self.threshold = threshold #cuttoff value to split
        self.left = left #points to the left child node
        self.right = right #points to the right child node
        self.value = value #value of the current node


#The building block for the forest, includes a structure for CART-style regression tree
#splits data by minimizing variance 
class DecisionTree:
    #Initializes with max depth and minimum number of samples per split 
    def __init__(self, max_depth=None, min_samples_split=2):
        self.root = None
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
    #Grow tree from the root node
    def fit(self, X, y):
        self.root = self._grow_tree(X,y)

    #grow tree based on the shape of X
    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        #If it surpasses max depth or the number of samples is below 2, create a new leaf node
        if depth >= self.max_depth or len(y) < self.min_samples_split or len(set(y)) == 1:
            return Node(value=np.mean(y))
        #Finding the best split
        #Set initial variance to infinity since any data will be less variance
        best_variance = float('inf')
        best_feature = best_threshold = None, None

        #Go through each feature in the tree and assign the features to left or right node based on threshold
        for feature in range(n_features):
            thresholds = sorted(set(X[:, feature]))
            for threshold in thresholds:
                left_indices = X[:, feature] <= threshold
                right_indices = X[:, feature] > threshold
                #Skip any invalid splits and compute variance
                if sum(left_indices) == 0 or sum(right_indices) == 0:
                    continue
                variance = self._calculate_variance(y[left_indices], y[right_indices])
                #Keep the best splits/features if their variance is minimized
                if variance < best_variance:
                    best_variance = variance
                    best_feature = feature
                    best_threshold = threshold
        #Recursively grow Tree
        #If there is no split found, then select current node as a leaf
        #If there is a split found, then continue to grow the tree as a root
        if best_feature is None or best_threshold is None:
            return Node(value=np.mean(y))

        left_indices = X[:, best_feature] <= best_threshold
        right_indices = X[:, best_feature] > best_threshold

        left = self._grow_tree(X[left_indices], y[left_indices], depth + 1)
        right = self._grow_tree(X[right_indices], y[right_indices], depth + 1)
        return Node(feature=best_feature, threshold=best_threshold, left=left, right=right)
    #Calculate variance
    def _calculate_variance(self, left_value, right_value):
        var_left = np.var(left_value)
        var_right = np.var(right_value)
        return (len(left_value) * var_left + len(right_value) * var_right) / (len(left_value) + len(right_value))

    #Predict sample by traversing the tree
    def predict(self, X):
        return [self._traverse_tree(x, self.root) for x in X]
    #Traverses tree from left to right, pre-order traversal
    def _traverse_tree(self, x, node):
        if node.value is not None:
            return node.value
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        else:
            return self._traverse_tree(x, node.right)

In [16]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

#Load the dataset
data = load_diabetes()
X, y = data.data, data.target

In [17]:
#Split the dataset into train and test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

In [18]:
#Initialize the random forest model on the training dataset
#Since it uses a custom build random forest, takes much longer than sklearn built-in forest:
#~5min for max_depth=5
random_forest = RandomForest(n_estimator=100, max_depth=5, min_samples_split=2)
random_forest.fit(X_train, y_train)

#Predict the values of the test dataset using random forest
predictions = random_forest.predict(X_test)

In [20]:
#Quanitfy the random forest's ability to predict test dataset
mse = mean_squared_error(y_test, predictions)
print(f"Mean squared error: {mse}")
print(y_train.min(), y_train.max(), y_train.mean())

Mean squared error: 2870.998742704578
25.0 346.0 153.73654390934846


In [60]:
#Now, we can use sklearn random forest model as well
#Regressor is used for continuous values and classifier is used for discrete 
#Completed in ~0.1sec with max_depth=5. The same value for mse, which means original structure was correct 
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import mean_squared_error

sk_rf = RandomForestRegressor(n_estimators=500, max_depth=3, random_state=42) 
sk_rf.fit(X_train, y_train) 
sk_predictions = sk_rf.predict(X_test) 
sk_mse = mean_squared_error(y_test, sk_predictions) 
print(f"Mean squared error: {sk_mse}")

Mean squared error: 2783.4670176611653


In [63]:
from sklearn.model_selection import GridSearchCV

#Define grid of parameters to test
param_grid = {
    'n_estimators': [100, 300, 500],
    'max_depth': [3, 5, 7, 10, None]
}

rf = RandomForestRegressor(random_state=42) 

grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',  # sklearn maximizes score, so we use negative MSE
    cv=5,
    n_jobs=-1
)

# Fit on training data
grid_search.fit(X_train, y_train)

# Best parameters and corresponding MSE
best_params = grid_search.best_params_
best_mse = -grid_search.best_score_

print("Best parameters:", best_params)
print("Best CV mean squared error:", best_mse)

# Train final model with best parameters
best_rf = RandomForestRegressor(**best_params, random_state=42)
best_rf.fit(X_train, y_train)
y_pred = best_rf.predict(X_test)
test_mse = mean_squared_error(y_test, y_pred)
print("Test set MSE:", test_mse)

Best parameters: {'max_depth': 5, 'n_estimators': 500}
Best CV mean squared error: 3380.2371224004464
Test set MSE: 2863.96523655835


In [57]:
import sklearn

sklearn.metrics.get_scorer_names()


['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_negative_likelihood_ratio',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'positive_likelihood_ratio',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',